In [1]:
import torch
from model import make_model
from gluonnlp.data import SentencepieceTokenizer
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model
from utils import greedy_decode

In [44]:
num_decoder_layers = 6
model_path = './output/step_26001.pth'

In [45]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model, vocab = make_model(num_decoder_layers)
model.load_state_dict(torch.load(model_path, map_location=device)['model'])
model.eval()
sp  = SentencepieceTokenizer(get_tokenizer())

using cached model
using cached model
using cached model


In [50]:
question = '사랑해?'
max_len = 20

tokens = sp(question)
tokens = ['[CLS]'] + tokens + ['[SEP]']
token_ids = [vocab.token_to_idx[tok] for tok in tokens]
# unsqueeze(0) for Batch position (zzingae)
token_ids = torch.tensor(token_ids).unsqueeze(0)
# attention score: [Batch, Head, tgt_length, src_length] in src_attn (zzingae)
# unsqueeze(1) for tgt_length position (zzingae)
attn_mask = (token_ids != vocab.token_to_idx['[PAD]']).unsqueeze(1).long()

answer = greedy_decode(model, token_ids, attn_mask, max_len, vocab)

with open('QnA_examples.txt','a',encoding='utf-8') as txt:
    Q = 'question: {}'.format(sp(question))
    A = 'answer: '+''.join([vocab.idx_to_token[idx] for idx in answer[0,1:]])
    txt.write(Q+'\n'+A+'\n')
    print(Q)
    print(A)

question: ['▁사랑', '해', '?']
answer: ▁당신이요.
